Hard sudoku solving techniques are based on getting the possibilities for each box, and analyzing those possibilities to exclude numbers from certain cells

In [1]:
# imports
from itertools import permutations 
import numpy as np
import sys
import math
import time
from solverFunctions import *

In [2]:
# test grids
emptySudoku = np.zeros((9,9))

hardSudoku = np.zeros((9,9))
hardSudoku[0,1] = 4
hardSudoku[0,7] = 1
hardSudoku[1,0] = 2
hardSudoku[1,8] = 6
hardSudoku[7,0] = 9
hardSudoku[7,8] = 2
hardSudoku[8,1] = 1
hardSudoku[8,7] = 9

sampleSudoku = np.array([[5, 3, 0, 0, 7, 0, 0, 0, 0],
                         [6, 0, 0, 1, 9, 5, 0, 0, 0],
                         [0, 9, 8, 0, 0, 0, 0, 6, 0],
                         [8, 0, 0, 0, 6, 0, 0, 0, 3],
                         [4, 0, 0, 8, 0, 3, 0, 0, 1],
                         [7, 0, 0, 0, 2, 0, 0, 0, 6],
                         [0, 6, 0, 0, 0, 0, 2, 8, 0],
                         [0, 0, 0, 4, 1, 9, 0, 0, 5],
                         [0, 0, 0, 0, 8, 0, 0, 7, 9]])

In [9]:
#loop through the cells to get all possible numbers in that cell
b = sampleSudoku.copy()
possibleNs = []
for ir in range (0,9):
    for ic in range(0,9):
        cellPoss = [0]
        for i in range(1,10):
            if probabilityFunction(b, ir, ic, i):
                if cellPoss[0] == 0:
                    cellPoss = [i]
                else:
                    cellPoss.append(i)
            if b[ir, ic] == i:
                cellPoss = i
        possibleNs.append(cellPoss)

81


## Naked pairs in a box

In [82]:
def nakedPairs(listPoss, rIdx):
    keepLooping = False
    for ir in rIdx:
        for intIr in (0,1):
            if len(listPoss[ir[intIr]]) > 1 and len(listPoss[ir[intIr+1]]) > 1:
                diffList = list(set(listPoss[ir[intIr]]).intersection(set(listPoss[ir[intIr+1]])))
                if len(diffList) == 2:
                    listPoss[ir[intIr]] = diffList
                    listPoss[ir[intIr+1]] = diffList
                    keepLooping = True
                 
                    for num in range(1,10):
                        availability = 0
                        for thisPoss in listPoss:
                            if num in thisPoss:
                                availability = availability + 1
                        if availability == 1:
                            
                            for thisPoss in range(0, len(listPoss)):
                                if num in listPoss[thisPoss]:
                                    listPoss[thisPoss] = [num]
    return listPoss, keepLooping

rowsIdx = [0, 1, 2, 9, 10, 11, 18, 20]
listPoss = []
listPoss.append(possibleNs[0])
listPoss.append(possibleNs[1])
listPoss.append(possibleNs[2])
listPoss.append(possibleNs[9])
listPoss.append(possibleNs[10])
listPoss.append(possibleNs[11])
listPoss.append(possibleNs[18])
listPoss.append(possibleNs[19])
listPoss.append(possibleNs[20])

rIdx = [[0,1,2],[3,4,5],[6,7,8]]
cIdx = [[0,3,6],[1,4,7],[2,5,8]]

keepLooping = True
while keepLooping:
    
    listPoss, keepLoopingR =  nakedPairs(listPoss, rIdx)
    listPoss, keepLoopingC =  nakedPairs(listPoss, cIdx)
    if keepLoopingC and keepLoopingR:
        keepLooping = True
    else :
        keepLooping = False
    
print(listPoss)

possibleNs[0] = listPoss[0]
possibleNs[1] = listPoss[1]
possibleNs[2] = listPoss[2]
possibleNs[9] = listPoss[3]
possibleNs[10] = listPoss[4]
possibleNs[11] = listPoss[5]
possibleNs[18] = listPoss[6]
possibleNs[19] = listPoss[7]
possibleNs[20] = listPoss[8]

# assign and note the cells with only 1 poss
idx = 0
for ir in range(0,9):
    for ic in range(0,9):
        if not possibleNs[idx][0] == 0 and len(possibleNs[idx]) == 1:
            b[ir, ic] = possibleNs[idx][0]
        idx = idx + 1


[[0], [0], [2, 4], [0], [7], [2, 4], [1], [0], [0]]
[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 0 0 8 0 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 0 2 8 0]
 [0 0 0 4 1 9 0 0 5]
 [0 0 0 0 8 0 0 7 9]]
[[5 3 0 0 7 0 0 0 0]
 [6 7 0 1 9 5 0 0 0]
 [1 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 0 0 8 5 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 7 2 8 4]
 [0 0 0 4 1 9 0 3 5]
 [0 0 0 0 8 0 0 7 9]]


In [114]:
#loop through the cells to get all possible numbers in that cell
b = hardSudoku.copy()
squareIdx = 8

possibleNs = []
for ir in range (0,9):
    for ic in range(0,9):
        cellPoss = [0]
        for i in range(1,10):
            if probabilityFunction(b, ir, ic, i):
                if cellPoss[0] == 0:
                    cellPoss = [i]
                else:
                    cellPoss.append(i)
        possibleNs.append(cellPoss)

        
allIdx = np.array(list(range(0,81))).reshape((9,9))
topLeftIdx = [[0, 0],
              [0, 3],
              [0, 6],
              [3, 0],
              [3, 3],
              [3, 6],
              [6, 0],
              [6, 3],
              [6, 6]]

allSquareIdxs = np.ndarray.flatten(allIdx[topLeftIdx[squareIdx][0]:topLeftIdx[squareIdx][0]+3,
           topLeftIdx[squareIdx][1]:topLeftIdx[squareIdx][1]+3])


listPoss = []
for i in allSquareIdxs:
    listPoss.append(possibleNs[i])

rIdx = [[0,1,2],[3,4,5],[6,7,8]]
cIdx = [[0,3,6],[1,4,7],[2,5,8]]

keepLooping = True
while keepLooping:
    
    listPoss, keepLoopingR =  nakedPairs(listPoss, rIdx)
    listPoss, keepLoopingC =  nakedPairs(listPoss, cIdx)
    if keepLoopingC and keepLoopingR:
        keepLooping = True
    else :
        keepLooping = False

for i in range(0, len(listPoss)):
    possibleNs[allSquareIdxs[i]] = listPoss[i]
    
# assign and note the cells with only 1 poss
idx = 0
for ir in range(0,9):
    for ic in range(0,9):
        if not possibleNs[idx][0] == 0 and len(possibleNs[idx]) == 1:
            b[ir, ic] = possibleNs[idx][0]
        idx = idx + 1
print(b)


[[0. 4. 0. 0. 0. 0. 0. 1. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 6.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [9. 0. 0. 0. 0. 0. 0. 0. 2.]
 [0. 1. 0. 0. 0. 0. 0. 9. 0.]]


In [116]:
squareIdx = 0
allSquareIdxs = np.ndarray.flatten(allIdx[topLeftIdx[squareIdx][0]:topLeftIdx[squareIdx][0]+3,
           topLeftIdx[squareIdx][1]:topLeftIdx[squareIdx][1]+3])


listPoss = []
for i in allSquareIdxs:
    listPoss.append(possibleNs[i])
print(listPoss)

squareIdx = 2
allSquareIdxs = np.ndarray.flatten(allIdx[topLeftIdx[squareIdx][0]:topLeftIdx[squareIdx][0]+3,
           topLeftIdx[squareIdx][1]:topLeftIdx[squareIdx][1]+3])


listPoss = []
for i in allSquareIdxs:
    listPoss.append(possibleNs[i])
print(listPoss)

[[3, 5, 6, 7, 8], [0], [3, 5, 6, 7, 8, 9], [0], [3, 5, 7, 8, 9], [1, 3, 5, 7, 8, 9], [1, 3, 5, 6, 7, 8], [3, 5, 6, 7, 8, 9], [1, 3, 5, 6, 7, 8, 9]]
[[2, 3, 5, 7, 8, 9], [0], [3, 5, 7, 8, 9], [3, 4, 5, 7, 8, 9], [3, 4, 5, 7, 8], [0], [2, 3, 4, 5, 7, 8, 9], [2, 3, 4, 5, 7, 8], [3, 4, 5, 7, 8, 9]]
